# Dataset Analysis

Loading Dataset

In [ ]:
def load_dataset(path):
    """
    
    """

Expolring Dataset

In [ ]:
def explore_dataset(df):
    """
    
    """

Preparing Dataset

- Images preparing

In [ ]:
def preprocess_images(df):
    """
    
    """

- labels preparing:

In [ ]:
def preprocess_labels(df):
    """
    
    """

In [ ]:
def prepare_dataset(df):
    """
    
    """
    images = preprocess_images(df)
    labels = preprocess_labels(df)
    return images, labels

# Data modeling

## Model Archeticticure:

- Base model

In [ ]:
def feature_extraction():
    """
    
    """

- Recognition model

In [ ]:
def reconition_model(featurres):
    """
    
    """

## Model Training and testing:

## Model Evaluaion metrics:

# User Experinece

## Image Accuisition.

In [ ]:
def input_image_preprocesisng(image):
    """
    
    """